# 2 - Visualization

Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

The Euclidean coordinates are derive from latitude, longitude and depth, which are define with respect to a reference ellipsoid.

A reference ellipsoid is a mathematically defined surface that approximates the geoid. It is used as a preferred surface on which geodetic network computations are performed and point coordinates such as latitude, longitude, and elevation (or depth) are defined.

For this purpose it is necessary to identify a zero meridian, which for Earth is usually the Prime Meridian.

$\lambda$ = longitude measures the rotational angle between the zero meridian and the measured point. By convention it is expressed in degrees ranging from $−180°$ to $+180°$.

$\phi$ = latitude measures how close to the poles or equator a point is along a meridian, and is represented as an angle from $−90°$ to $+90°$, where $0°$ is the equator. 

$h$ = ellipsoidal height of the point above or below the reference ellipsoid along its normal.

If these coordinates are given, one can compute the geocentric rectangular coordinates of the point as follows:

\begin{aligned}X&={\big (}N(\phi )+h{\big )}\cos {\phi }\cos {\lambda }\\Y&={\big (}N(\phi )+h{\big )}\cos {\phi }\sin {\lambda }\\Z&=\left({\frac {b^{2}}{a^{2}}}N(\phi )+h\right)\sin {\phi }\end{aligned}
where

$$N(\phi )= \frac {a^{2}} {\sqrt {a^{2}\cos ^{2}\phi +b^{2}\sin ^{2}\phi}}$$
and $a$ and $b$ are the equatorial radius (semi-major axis) and the polar radius (semi-minor axis), respectively. $N$ is the radius of curvature in the prime vertical.


In [2]:
Image(url='https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/WGS84_mean_Earth_radius.svg/440px-WGS84_mean_Earth_radius.svg.png')

#Image(url='https://www.e-education.psu.edu/geog862/sites/www.e-education.psu.edu.geog862/files/images/Lesson06/Ellipsoidal%20Heights.png')

#Image(url='https://www.e-education.psu.edu/geog862/sites/www.e-education.psu.edu.geog862/files/images/Lesson06/Ellipsoidal%20Heights3.png')

In [3]:
# Read data from file and make sure earthquakes are orderd by time

df = pd.read_table("SouthCalifornia-1982-2011_Physics-of-Data.dat",sep=" ")

In [4]:
from pygeodesy.ellipsoidalVincenty import LatLon, Cartesian # Package used to tranform cartesian coordinate into geographic coordinate

hw_pos = [-2507270,-4652320,3558770] # Hollywood position used as georeference
df_min = np.array(df.loc[df['depth'] == df['depth'].min()])
df_max = np.array(df.loc[df['depth'] == df['depth'].max()])

pos_min = Cartesian(df_min[0,-3],df_min[0,-2],df_min[0,-1]).toLatLon()
pos_max = Cartesian(df_max[0,-3],df_max[0,-2],df_max[0,-1]).toLatLon()
pos_hollywood = Cartesian(hw_pos[0],hw_pos[1],hw_pos[2]).toLatLon()


d = np.empty(df.shape[0], object)

for i in range(d.shape[0]):
    d[i] = Cartesian(np.array(df.iloc[i,-3]), np.array(df.iloc[i,-2]), np.array(df.iloc[i,-1])).toLatLon()

print("Position of shallowest earthquake: ", pos_max)
print("Position of deepest earthquake: ", pos_min)
print("Position of Hollywood: ", pos_hollywood)

ModuleNotFoundError: No module named 'pygeodesy'

In [ ]:
print(d)

In [ ]:
# Plot of all earthquakes with Hollywood reference
plt.plot(df['latitude'],df['longitude'],'.')
plt.plot(-2507270,-4652320,'o')
plt.show()

In [ ]:
m = np.linspace(2,7.3,32)
number = []

df = pd.read_table("SouthCalifornia-1982-2011_Physics-of-Data.dat",sep=" ")
df = df.sort_values(by = ['time'])

for i in m:    
    df_red = df[df['magnitude'] >= i]
    number.append(df_red.shape[0])

fig, ax = plt.subplots(figsize=(10,10))

plt.plot(m,number,'.')

plt.xscale('log')
#plt.yscale('log')

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.animation as animation
from IPython.display import HTML

In [6]:
df = pd.read_table("SouthCalifornia-1982-2011_Physics-of-Data.dat",sep=" ")
df = df.sort_values(by = ['time'])

In [5]:
import math 

R = 6371005

def xyz_to_latlon (x,y,z):
    """Convert cartesian to angular lat/lon coordiantes"""
    r = math.sqrt(x**2 + y**2 + z**2)
    rad = np.abs(r - R)
    theta = math.asin(z/r) # https://stackoverflow.com/a/1185413/4933053
    phi = math.atan2(y,x)
    lat = math.degrees(theta)
    lon = math.degrees(phi)
    return lat, lon, rad

lat = np.empty(df.shape[0], object)
lon = np.empty(df.shape[0], object)
rad = np.empty(df.shape[0], object)

for i in range(df.shape[0]):
    lat[i],lon[i],rad[i] = xyz_to_latlon(np.array(df.iloc[i,-3]), np.array(df.iloc[i,-2]), np.array(df.iloc[i,-1]))

df['latitude'] = lat
df['longitude'] = lon
df['depth'] = rad 

In [6]:
import datetime
import time

#datetime.datetime.today()
#datetime.datetime.fromtimestamp(time.time())

#offset = datetime.datetime.fromisoformat('1982-01-01').strftime('%s') #.ctime()

offset = (datetime.datetime.fromisoformat('1982-01-01') - datetime.datetime.fromisoformat('1970-01-01')).total_seconds() 

#x = (datetime.datetime.fromisoformat('1987-01-01') - datetime.datetime.fromisoformat('1982-01-01')).total_seconds() + offset
#y = float( datetime.datetime.fromtimestamp(x).year )
#datetime.datetime.now()

df['total_second'] = df['time'] + offset 

#df['year'] = datetime.datetime.fromtimestamp( df['total_second'] ).year 

y = np.empty(df['time'].values.shape[0], int)

for i in range(df['time'].values.shape[0]):
    y[i] = datetime.datetime.fromtimestamp( df['total_second'].values[i] ).year 

df['year'] = y
m=3
df = df[df['magnitude']>m]

KeyError: 'time'

In [9]:
df

,index,trigger,time,magnitude,latitude,longitude,depth,total_second,year
50,50,0,8.945939e+05,3.09,34.7845,-120.941,19665.1,3.795858e+08,1982
83,83,0,1.367108e+06,3.03,35.7795,-120.971,17174.7,3.800583e+08,1982
102,102,0,1.593043e+06,3.03,34.2649,-119.643,8826.37,3.802842e+08,1982
113,113,0,1.813600e+06,3.04,37.495,-119.409,9704.87,3.805048e+08,1982
121,121,0,1.916778e+06,3.92,37.4113,-117.903,16325.2,3.806080e+08,1982
...,...,...,...,...,...,...,...,...,...
110235,110235,-1,9.298340e+08,3.42,31.898,-114.932,11215.3,1.308525e+09,2011
110245,110245,-1,9.301223e+08,3.27,35.9213,-117.741,9123.47,1.308814e+09,2011
110248,110248,110247,9.301739e+08,3.08,33.1642,-115.659,5686.93,1.308865e+09,2011
110249,110249,100065,9.301998e+08,3.17,32.3054,-115.332,12277.9,1.308891e+09,2011


In [ ]:
#df_min = np.array(df.loc[df['depth'] == df['depth'].min()])
#pos_min = xyz_to_latlon(df_min[0,-3],df_min[0,-2],df_min[0,-1])
#print(pos_min)

In [7]:
fig = plt.figure(figsize=(10, 10))
#fig.text(.8, .3, 'Soumitra', ha='right')
m = Basemap(projection='mill',llcrnrlon=-125.97 , llcrnrlat=28.62, urcrnrlon=-111.95, urcrnrlat=38.41,resolution='c')

#m.drawcoastlines()
m.drawcountries()
#m.fillcontinents(color='white',lake_color='lightblue', zorder = 1)
#m.drawmapboundary(fill_color='lightblue')
m.shadedrelief()

START_YEAR = 1982
LAST_YEAR = 2011

points = df[df['year']==START_YEAR]

x, y = m(list(points['longitude']), list(points['latitude']))

mag = list( (np.exp(1.15*points['magnitude'])) )

color = list(points['depth'])

scat = m.scatter(x, y, s = mag,  marker='o', alpha=0.5, c = color, cmap = 'hot')  #s = points['magnitude']*points['depth']*0.3, marker='o', alpha=0.3, zorder=10, cmap = cmap)

year_text = plt.text(-170, 80, str(START_YEAR),fontsize=30)

plt.title("Earthquake visualisation (1982 - 2011)")

for magn in [2, 3, 4, 5, 6]:
    plt.scatter([], [], c='k', alpha=0.5, s=np.exp(1.15*magn),
                label=str(magn))
    
plt.legend(scatterpoints=1, frameon=False, labelspacing=2, title='Magnitude', loc="upper left")

plt.colorbar(scat,label = 'depth')
plt.clim(100000,120000)

plt.axis(aspect='equal')
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.close()

def update(frame_number):
    
    current_year = START_YEAR + (frame_number % (LAST_YEAR - START_YEAR ))
    
    year_text.set_text(str(current_year))
    points = df[df['year']==current_year]
    
    x, y = m(list(points['longitude']), list(points['latitude'])) 
    mag = list( (np.exp(1.15*points['magnitude'])) )
    color = list( points['depth'] )
    
    #print(color)
    #print('\ ')
    scat.set_offsets(np.c_[x,y]) #np.dstack((x, y)))
    scat.set_cmap('hot')
    #scat.set_array(color)
    scat.set_sizes(mag)
    
    
ani = animation.FuncAnimation(fig, update, interval=100, frames=LAST_YEAR - START_YEAR)

ani.save('animation.gif', writer='imagemagick', fps=1)    



import io
import base64

filename = 'animation.gif'

video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

NameError: name 'Basemap' is not defined

<Figure size 720x720 with 0 Axes>